In [1]:
from __future__ import absolute_import, division, print_function

import sys
import optparse
import csv
import re
import itertools as it

import h5py as h5

import logging

import numpy as np
import scipy as sp
import pandas as pd

import csi

In [2]:
import numpy as np
import scipy as sp
import scipy.optimize as spo
import scipy.stats as sps
import pandas as pd
import h5py as h5

import multiprocessing as mp

import itertools as it

import gp

In [3]:
inp = csi.loadData('testdata/Botrytis_Data_normalised.csv')

In [4]:
gpprior = [10,0.1]

In [5]:
genes = list(inp.index)

In [6]:
cc = csi.Csi(inp)

In [7]:
em = cc.getEm()

In [8]:
genes[0]

'AT4G34000'

In [9]:
em.setup(cc.allParents(genes[0],2))

In [14]:
sp.stats.gamma.pdf(1,a=10,scale=0.1)

1.2511003572113371

In [22]:
np.prod(sp.stats.gamma.pdf(sp.exp(sp.randn(3)),a=10,scale=0.1))

8.6820232741249312e-10

In [17]:
sp.log(3)

1.0986122886681098

In [18]:
a=1
b=2

In [19]:
(a,b) = (a,b)-np.max((a,b))

In [20]:
a

-1

In [21]:
b

0

In [27]:
em.set_priors(10,0.1)

In [30]:
em._prior_shape+3

13

In [29]:
em._prior_scale

array(0.1)

In [33]:
np.random.uniform()

0.3645131479687487

In [34]:
sp.randn()

-0.8796702306818074

In [36]:
73%57

16

In [40]:
em.hypers = sp.exp([-0.4491,0.1774,-0.1868])

In [41]:
em.pset = cc.allParents('AT4G34000',1)

In [42]:
em.logliks()

array([-254.65416263, -256.67022449, -252.51430866, -256.59080306,
       -235.4970021 , -251.3027252 , -242.71820458, -239.35989062,
       -244.41833429, -239.2181522 , -241.70916843, -245.34641921,
       -238.44528674, -243.27226345, -241.7483005 , -242.66771724,
       -241.72889515, -252.42949094, -242.48885364, -255.9714944 ,
       -247.98081618, -254.90309459, -255.34764501, -240.43599632,
       -254.88848964, -246.20110904, -243.87228624, -255.9883874 ,
       -256.83055056, -252.00006026, -244.40176847, -250.40037488,
       -245.83544469, -256.50745092, -245.40886994, -244.97884105,
       -239.74919108, -238.16303607, -242.25757587, -251.64193311,
       -238.80062715, -255.05460288, -246.00564274, -243.29174505,
       -239.35270813, -255.12073468, -243.37986838, -257.91321744,
       -251.58934633, -240.20004706, -245.08969362, -255.88726916,
       -244.62976457, -253.30053208, -238.01114207, -244.36518597,
       -242.13274995, -239.61515976, -255.43628246, -247.47162

In [43]:
import numpy.matlib as npm
import math

def loglik(X,Y,hypers):
	#hypers as [length, signal std, noise std] in log space
	thing = X / sp.exp(hypers[0])
	dist = np.zeros((len(Y),len(Y)))
	for i in np.arange(len(Y)):
		for j in np.arange(len(Y)):
			dist[i,j] = np.sum(np.square(thing[i,:]-thing[j,:]))
	K = sp.exp(2*hypers[1]+(-1/2)*dist)
	L = np.linalg.cholesky(K + sp.exp(2*hypers[2])*npm.eye(len(Y)))
	alpha = np.linalg.solve(np.transpose(L),np.linalg.solve(L,Y))
	out = (-0.5)*np.dot(np.transpose(Y),alpha)-np.sum(np.eye(len(Y))*L)-(X.shape[1]/2)*np.log(2*math.pi)
	return out

In [45]:
X = em.X[:,:2]

TypeError: unhashable type: 'slice'

In [46]:
em.X

AT4G34000  AT3G19290  AT2G40220  AT2G36270  AT1G01480  \
sample,replicate hours                                                          
Botrytis:A       2      -1.005128   0.131742   0.491480  -0.634663  -0.553422   
                 4      -0.901464   0.727480   0.487648   0.605025  -0.367126   
                 6       1.476983   1.318623  -0.925644   0.615760  -0.296208   
                 8       0.789920   0.008204  -0.452926   0.556027  -0.452172   
                 10      2.220758   0.542562  -0.567622   0.410010  -0.643295   
                 12     -0.742055   0.486430  -0.366723  -0.197133  -0.225244   
                 14     -0.698624   0.632142  -0.601311  -1.660394  -0.394703   
                 16      1.105240  -0.009641   1.216349   0.390508  -0.598818   
                 18      0.132847   0.129336  -0.150605  -0.450886  -0.750061   
                 20      1.366016   0.220571  -0.885360   1.113928  -1.075408   
                 22      0.292972   2.635230  -0.494566  -0.254652  -0.609860   
                 24      0.485219   0.404590   2.465125   0.375521  -0.579675   
                 26      0.006670   0.446896   0.892726  -0.171675  -1.041443   
                 28     -0.829932  -0.703055   1.529133   0.142198   0.078592   
                 30      1.738602   0.638971  -0.114106   0.359189  -0.865597   
                 32      1.313264  -0.097418  -0.351060  -0.332384  -0.751938   
                 34      0.083945  -0.280589  -0.804481   0.051813  -0.321327   
                 36     -0.477393   0.090701  -0.554374   0.536009  -0.576480   
                 38      2.177634   2.003104   0.061641   0.481684  -0.046673   
                 40      0.242574   0.654963  -0.626832   0.470467  -0.430387   
                 42      1.133351   0.570847  -0.485343   0.534318  -0.677429   
                 44      1.228888   0.187431  -1.132619   0.445798  -0.478190   
                 46      1.393030   0.549945  -1.580732   0.227463  -0.868053   
Botrytis:B       2      -1.003921   0.066419   1.030061  -0.277679  -0.884107   
                 4      -0.267215   0.605971  -1.057123   0.464547  -0.580061   
                 6       0.165645   0.264831  -0.509710   1.077768  -0.425345   
                 8      -0.095233  -1.370873  -0.839040  -4.314838  -1.195991   
                 10      0.148252  -0.331936   0.366360   0.736860  -0.383989   
                 12      0.035439   1.041481  -1.744034   0.019092  -0.430541   
                 14     -1.114506   0.050344  -0.873093  -1.034582  -1.141454   
...                           ...        ...        ...        ...        ...   
Mock:C           34     -0.583629  -0.705241   0.951077  -0.919976   1.315078   
                 36     -0.837824   0.203008  -1.391861   0.103357   0.707469   
                 38     -0.854144  -1.053210   0.704545   0.477735   1.483487   
                 40     -0.685294  -1.142034   0.040825  -0.188245   1.333197   
                 42     -1.316891  -0.152391   1.645528   0.244850   1.947836   
                 44     -0.544141  -1.637163   0.491135   0.504255   1.919988   
                 46     -1.306591  -0.737239   0.407031   0.176154   1.229720   
Mock:D           2      -1.136219   1.040786   1.075453  -1.600462  -0.406147   
                 4      -0.715827  -0.039095   0.557255  -0.195850  -0.247657   
                 6       1.682788   1.858123   0.220498  -0.781151  -0.197702   
                 8       1.403598   0.521354   0.619227   0.378603  -0.275711   
                 10      0.097377  -0.159891  -0.657865  -0.898869  -0.695649   
                 12     -0.091661   0.236883   0.863272   0.028644  -0.430561   
                 14     -0.322104   0.514984  -1.112172   0.114483  -0.635438   
                 16      0.467650   0.919307  -0.163467   1.025463  -0.825046   
                 18      0.564302   0.801348  -0.329377   0.271457   0.394120   
                 20      0.279039  -0.893018  -0.368949

In [47]:
kek = em.X.values

In [49]:
X=kek[:,:2]

In [50]:
Y=kek[:,0]

In [51]:
hypers = [0.1774,-0.4491,-0.1868]

In [52]:
import numpy.matlib as npm
import math

def loglik(X,Y,hypers):
	#hypers as [length, signal std, noise std] in log space
	thing = X / sp.exp(hypers[0])
	dist = np.zeros((len(Y),len(Y)))
	for i in np.arange(len(Y)):
		for j in np.arange(len(Y)):
			dist[i,j] = np.sum(np.square(thing[i,:]-thing[j,:]))
	K = sp.exp(2*hypers[1]+(-1/2)*dist)
	L = np.linalg.cholesky(K + sp.exp(2*hypers[2])*npm.eye(len(Y)))
	alpha = np.linalg.solve(np.transpose(L),np.linalg.solve(L,Y))
	out = (-0.5)*np.dot(np.transpose(Y),alpha)-np.sum(np.eye(len(Y))*L)-(X.shape[1]/2)*np.log(2*math.pi)
	return out

In [53]:
loglik(X,Y,kek)

ValueError: operands could not be broadcast together with shapes (184,2) (96,) 

In [54]:
X

array([[ -1.00512755e+00,   1.31742447e-01],
       [ -9.01463542e-01,   7.27479780e-01],
       [  1.47698288e+00,   1.31862292e+00],
       [  7.89919638e-01,   8.20369400e-03],
       [  2.22075850e+00,   5.42561601e-01],
       [ -7.42055238e-01,   4.86429770e-01],
       [ -6.98624155e-01,   6.32142345e-01],
       [  1.10523980e+00,  -9.64059100e-03],
       [  1.32846998e-01,   1.29335948e-01],
       [  1.36601588e+00,   2.20570777e-01],
       [  2.92972427e-01,   2.63523025e+00],
       [  4.85219159e-01,   4.04589518e-01],
       [  6.67013800e-03,   4.46896105e-01],
       [ -8.29931730e-01,  -7.03054659e-01],
       [  1.73860222e+00,   6.38971075e-01],
       [  1.31326415e+00,  -9.74182110e-02],
       [  8.39450150e-02,  -2.80589158e-01],
       [ -4.77393008e-01,   9.07014960e-02],
       [  2.17763375e+00,   2.00310396e+00],
       [  2.42573772e-01,   6.54963398e-01],
       [  1.13335053e+00,   5.70847207e-01],
       [  1.22888769e+00,   1.87430998e-01],
       [  

In [55]:
X/sp.exp(hypers[0])

array([[ -8.41738781e-01,   1.10327019e-01],
       [ -7.54925901e-01,   6.09224115e-01],
       [  1.23689155e+00,   1.10427383e+00],
       [  6.61514046e-01,   6.87014039e-03],
       [  1.85976252e+00,   4.54365359e-01],
       [ -6.21430256e-01,   4.07358052e-01],
       [ -5.85059124e-01,   5.29384282e-01],
       [  9.25577257e-01,  -8.07346222e-03],
       [  1.11252020e-01,   1.08311709e-01],
       [  1.14396282e+00,   1.84715837e-01],
       [  2.45348218e-01,   2.20685971e+00],
       [  4.06344232e-01,   3.38821364e-01],
       [  5.58587198e-03,   3.74250792e-01],
       [ -6.95021962e-01,  -5.88769426e-01],
       [  1.45598328e+00,   5.35102966e-01],
       [  1.09978616e+00,  -8.15823683e-02],
       [  7.02993112e-02,  -2.34977914e-01],
       [ -3.99790263e-01,   7.59574907e-02],
       [  1.82364793e+00,   1.67748888e+00],
       [  2.03142129e-01,   5.48495652e-01],
       [  9.49118438e-01,   4.78052991e-01],
       [  1.02912553e+00,   1.56963103e-01],
       [  

In [56]:
thing = X / sp.exp(hypers[0])

In [57]:
dist = np.zeros((len(Y),len(Y)))

In [58]:
	for i in np.arange(len(Y)):
		for j in np.arange(len(Y)):
			dist[i,j] = np.sum(np.square(thing[i,:]-thing[j,:]))

In [59]:
K = sp.exp(2*hypers[1]+(-1/2)*dist)

In [60]:
L = np.linalg.cholesky(K + sp.exp(2*hypers[2])*npm.eye(len(Y)))

In [61]:
alpha = np.linalg.solve(np.transpose(L),np.linalg.solve(L,Y))

In [62]:
out = (-0.5)*np.dot(np.transpose(Y),alpha)-np.sum(np.eye(len(Y))*L)-(X.shape[1]/2)*np.log(2*math.pi)

In [63]:
out

-637.34302766904023

In [64]:
K

array([[ 0.40730214,  0.35828827,  0.02865243, ...,  0.08631601,
         0.31334318,  0.39682738],
       [ 0.35828827,  0.40730214,  0.04956821, ...,  0.03239204,
         0.2613937 ,  0.31908324],
       [ 0.02865243,  0.04956821,  0.40730214, ...,  0.00171264,
         0.07358727,  0.02884619],
       ..., 
       [ 0.08631601,  0.03239204,  0.00171264, ...,  0.40730214,
         0.11517981,  0.12217578],
       [ 0.31334318,  0.2613937 ,  0.07358727, ...,  0.11517981,
         0.40730214,  0.34473654],
       [ 0.39682738,  0.31908324,  0.02884619, ...,  0.12217578,
         0.34473654,  0.40730214]])

In [65]:
L

matrix([[  1.04668730e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  3.42306886e-01,   9.89131082e-01,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  2.73743899e-02,   4.06394796e-02,   1.04553975e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        ..., 
        [  8.24658996e-02,   4.20914721e-03,  -6.84691257e-04, ...,
           8.67846326e-01,   0.00000000e+00,   0.00000000e+00],
        [  2.99366565e-01,   1.60664718e-01,   5.62991122e-02, ...,
          -3.57173748e-03,   8.38588053e-01,   0.00000000e+00],
        [  3.79126964e-01,   1.91385619e-01,   1.02243889e-02, ...,
          -4.93063577e-03,   9.38922335e-03,   8.40962614e-01]])

In [70]:
np.eye(184)*L

matrix([[  1.04668730e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  3.42306886e-01,   9.89131082e-01,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  2.73743899e-02,   4.06394796e-02,   1.04553975e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        ..., 
        [  8.24658996e-02,   4.20914721e-03,  -6.84691257e-04, ...,
           8.67846326e-01,   0.00000000e+00,   0.00000000e+00],
        [  2.99366565e-01,   1.60664718e-01,   5.62991122e-02, ...,
          -3.57173748e-03,   8.38588053e-01,   0.00000000e+00],
        [  3.79126964e-01,   1.91385619e-01,   1.02243889e-02, ...,
          -4.93063577e-03,   9.38922335e-03,   8.40962614e-01]])

In [68]:
np.eye(6)

array([[ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.]])

In [71]:
em.pset

[([], 'AT4G34000'),
 (['AT3G19290'], 'AT4G34000'),
 (['AT2G40220'], 'AT4G34000'),
 (['AT2G36270'], 'AT4G34000'),
 (['AT1G01480'], 'AT4G34000'),
 (['AT2G37620'], 'AT4G34000'),
 (['AT3G01420'], 'AT4G34000'),
 (['AT1G52890'], 'AT4G34000'),
 (['AT1G61110'], 'AT4G34000'),
 (['AT3G56530'], 'AT4G34000'),
 (['AT5G13180'], 'AT4G34000'),
 (['AT5G39610'], 'AT4G34000'),
 (['AT5G63790'], 'AT4G34000'),
 (['AT5G62000'], 'AT4G34000'),
 (['AT3G51910'], 'AT4G34000'),
 (['AT1G06160'], 'AT4G34000'),
 (['AT1G25550'], 'AT4G34000'),
 (['AT1G35560'], 'AT4G34000'),
 (['AT1G79320'], 'AT4G34000'),
 (['AT2G18300'], 'AT4G34000'),
 (['AT2G24430'], 'AT4G34000'),
 (['AT2G28510'], 'AT4G34000'),
 (['AT2G40340'], 'AT4G34000'),
 (['AT2G45220'], 'AT4G34000'),
 (['AT2G45800'], 'AT4G34000'),
 (['AT3G13040'], 'AT4G34000'),
 (['AT3G23220'], 'AT4G34000'),
 (['AT3G28320'], 'AT4G34000'),
 (['AT3G57600'], 'AT4G34000'),
 (['AT3G58120'], 'AT4G34000'),
 (['AT4G14410'], 'AT4G34000'),
 (['AT4G26410'], 'AT4G34000'),
 (['AT5G08330'], 'A

In [72]:
em.hypers = sp.exp([-0.4139,0.0555,-0.2025])

In [73]:
em.logliks()

array([-255.90502949, -257.89435457, -253.12071982, -258.26755167,
       -236.07950355, -252.71726611, -243.5383114 , -239.71830979,
       -245.52635833, -240.16844542, -242.87979049, -246.72314599,
       -238.72917114, -243.80854996, -242.6044852 , -243.20516674,
       -242.14811032, -254.33796853, -243.56376466, -257.28375155,
       -249.22318643, -256.24114627, -256.73293142, -241.0392226 ,
       -256.33290448, -247.34876392, -244.90020557, -257.45894938,
       -258.41323529, -253.21476851, -244.68893341, -252.00736468,
       -246.75474085, -257.81323173, -246.31130545, -246.01781705,
       -240.21908305, -239.14608149, -243.44044978, -253.07732389,
       -239.47997266, -256.91264519, -247.36980975, -244.49161888,
       -239.96825585, -256.72111726, -244.44905051, -259.52118207,
       -252.91726892, -241.19448459, -245.62007245, -257.52463593,
       -245.6483891 , -254.83990927, -238.92870412, -245.1764905 ,
       -242.51619914, -240.87631861, -256.47796748, -248.40952

In [75]:
loglik(X,Y,[0.0555,-0.4139,-0.2025])

-624.04393471572712

In [78]:
numtemp = np.arange(len(em.logliks()))

In [80]:
numtemp[em.logliks()==np.min(em.logliks())]

array([47])

In [81]:
kek = np.zeros(5)

In [82]:
kek

array([ 0.,  0.,  0.,  0.,  0.])

In [88]:
kek2 = np.ones((5,1))

In [89]:
kek2

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]])

In [90]:
kek*kek2

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [91]:
np.dot(kek,kek2)

array([ 0.])

In [92]:
kek2*kek

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [93]:
npm.eye(5)*7

matrix([[ 7.,  0.,  0.,  0.,  0.],
        [ 0.,  7.,  0.,  0.,  0.],
        [ 0.,  0.,  7.,  0.,  0.],
        [ 0.,  0.,  0.,  7.,  0.],
        [ 0.,  0.,  0.,  0.,  7.]])

In [94]:
kek2

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]])

In [98]:
np.sum(np.dot(kek2,np.transpose(kek2)))

25.0

In [99]:
import numpy.matlib as npm
import math

def loglik(X,Y,hypers):
	#hypers as [length, signal std, noise std] in log space
	thing = X / sp.exp(hypers[0])
	dist = np.zeros((len(Y),len(Y)))
	for i in np.arange(len(Y)):
		for j in np.arange(len(Y)):
			dist[i,j] = np.sum(np.square(thing[i,:]-thing[j,:]))
	K = sp.exp(2*hypers[1]+(-1/2)*dist)
	L = np.linalg.cholesky(K + sp.exp(2*hypers[2])*npm.eye(len(Y)))
	alpha = np.linalg.solve(np.transpose(L),np.linalg.solve(L,Y))
	out = (-0.5)*np.dot(np.transpose(Y),alpha)-np.sum(np.eye(len(Y))*L)-(X.shape[0]/2)*np.log(2*math.pi)
	return out

In [100]:
loglik(X,Y,hypers)

-804.58984071229065

In [101]:
def loglik(X,Y,hypers):
	#hypers as [length, signal std, noise std] in log space
	thing = X / sp.exp(hypers[0])
	dist = np.zeros((len(Y),len(Y)))
	for i in np.arange(len(Y)):
		for j in np.arange(len(Y)):
			dist[i,j] = np.sum(np.square(thing[i,:]-thing[j,:]))
	K = sp.exp(2*hypers[1]+(-1/2)*dist)
	L = np.linalg.cholesky(K + sp.exp(2*hypers[2])*npm.eye(len(Y)))
	alpha = np.linalg.solve(np.transpose(L),np.linalg.solve(L,Y))
	out = (-0.5)*np.dot(np.transpose(Y),alpha)-np.sum(np.eye(len(Y))*np.log(L))-(X.shape[0]/2)*np.log(2*math.pi)
	return out

In [102]:
loglik(X,Y,hypers)

nan

In [104]:
wot=np.array([[25,15,-5],[15,18,0],[-5,0,11]])

In [105]:
np.linalg.cholesky(wot)

array([[ 5.,  0.,  0.],
       [ 3.,  3.,  0.],
       [-1.,  1.,  3.]])

In [106]:
wot

array([[25, 15, -5],
       [15, 18,  0],
       [-5,  0, 11]])

In [107]:
Y=np.array([1,3,5])

In [108]:
Y

array([1, 3, 5])

In [109]:
np.transpose(Y)

array([1, 3, 5])

In [114]:
Y=X[:,0]

In [115]:
Y

array([-1.00512755, -0.90146354,  1.47698288,  0.78991964,  2.2207585 ,
       -0.74205524, -0.69862415,  1.1052398 ,  0.132847  ,  1.36601588,
        0.29297243,  0.48521916,  0.00667014, -0.82993173,  1.73860222,
        1.31326415,  0.08394501, -0.47739301,  2.17763375,  0.24257377,
        1.13335053,  1.22888769,  1.39302993, -1.00392142, -0.26721549,
        0.1656454 , -0.09523287,  0.14825173,  0.03543933, -1.11450581,
       -0.58581641, -0.13449733,  0.44627338, -0.2589981 ,  0.74985539,
       -0.1667758 ,  0.95696289,  1.13200207,  0.18265187,  0.77900653,
       -1.31101942, -0.30647343,  0.66021172,  1.46187731,  0.57554358,
        1.069807  ,  0.97905817,  0.32556715, -1.80868844,  0.92131538,
        2.0673161 ,  0.88286535, -0.84587228,  0.15672917,  0.39274177,
        2.74954867,  0.56185064,  0.75602726,  0.42006771,  1.05668675,
        0.41173727,  0.00367855, -0.9812741 , -0.63889386, -0.14835498,
       -0.35939411,  1.40286978,  2.05120869,  1.29788442, -0.21

In [116]:
Y.shape = (len(Y),1)

In [117]:
loglik(X,Y,hypers)

array([[ nan]])

In [118]:
Y

array([[-1.00512755],
       [-0.90146354],
       [ 1.47698288],
       [ 0.78991964],
       [ 2.2207585 ],
       [-0.74205524],
       [-0.69862415],
       [ 1.1052398 ],
       [ 0.132847  ],
       [ 1.36601588],
       [ 0.29297243],
       [ 0.48521916],
       [ 0.00667014],
       [-0.82993173],
       [ 1.73860222],
       [ 1.31326415],
       [ 0.08394501],
       [-0.47739301],
       [ 2.17763375],
       [ 0.24257377],
       [ 1.13335053],
       [ 1.22888769],
       [ 1.39302993],
       [-1.00392142],
       [-0.26721549],
       [ 0.1656454 ],
       [-0.09523287],
       [ 0.14825173],
       [ 0.03543933],
       [-1.11450581],
       [-0.58581641],
       [-0.13449733],
       [ 0.44627338],
       [-0.2589981 ],
       [ 0.74985539],
       [-0.1667758 ],
       [ 0.95696289],
       [ 1.13200207],
       [ 0.18265187],
       [ 0.77900653],
       [-1.31101942],
       [-0.30647343],
       [ 0.66021172],
       [ 1.46187731],
       [ 0.57554358],
       [ 1

In [119]:
Y=np.array([1,3,5])

In [120]:
Y.shape = (3,1)

In [121]:
Y

array([[1],
       [3],
       [5]])

In [122]:
L

array([[25, 15, -5],
       [15, 18,  0],
       [-5,  0, 11]])

In [127]:
np.linalg.solve(wot,np.linalg.solve(np.transpose(wot),Y))

array([[ 0.02066612],
       [-0.01146045],
       [ 0.05383813]])

In [125]:
L

array([[25, 15, -5],
       [15, 18,  0],
       [-5,  0, 11]])

In [126]:
np.transpose(L)

array([[25, 15, -5],
       [15, 18,  0],
       [-5,  0, 11]])

In [128]:
wot

array([[25, 15, -5],
       [15, 18,  0],
       [-5,  0, 11]])

In [130]:
wot = np.linalg.cholesky(L)

In [131]:
wot

array([[ 5.,  0.,  0.],
       [ 3.,  3.,  0.],
       [-1.,  1.,  3.]])

In [133]:
np.linalg.solve(np.transpose(wot),np.linalg.solve(wot,Y))

array([[ 0.07555556],
       [ 0.1037037 ],
       [ 0.48888889]])

In [134]:
L

array([[25, 15, -5],
       [15, 18,  0],
       [-5,  0, 11]])

In [135]:
X=L

In [136]:
Y=X

In [137]:
	thing = X / sp.exp(hypers[0])
	dist = np.zeros((len(Y),len(Y)))
	for i in np.arange(len(Y)):
		for j in np.arange(len(Y)):
			dist[i,j] = np.sum(np.square(thing[i,:]-thing[j,:]))

In [138]:
dist

array([[   0.        ,   93.97603472,  968.51420854],
       [  93.97603472,    0.        ,  592.61006967],
       [ 968.51420854,  592.61006967,    0.        ]])

In [139]:
X

array([[25, 15, -5],
       [15, 18,  0],
       [-5,  0, 11]])

In [140]:
thing = X

In [141]:
	dist = np.zeros((len(Y),len(Y)))
	for i in np.arange(len(Y)):
		for j in np.arange(len(Y)):
			dist[i,j] = np.sum(np.square(thing[i,:]-thing[j,:]))

In [142]:
dist

array([[    0.,   134.,  1381.],
       [  134.,     0.,   845.],
       [ 1381.,   845.,     0.]])

In [145]:
K = sp.exp(2*1+(-1/2)*dist)

In [146]:
K

array([[  7.38905610e+000,   5.90009054e-029,   9.73305572e-300],
       [  5.90009054e-029,   7.38905610e+000,   2.39425476e-183],
       [  9.73305572e-300,   2.39425476e-183,   7.38905610e+000]])

In [147]:
dist

array([[    0.,   134.,  1381.],
       [  134.,     0.,   845.],
       [ 1381.,   845.,     0.]])

In [148]:
L = np.linalg.cholesky(K + sp.exp(2*hypers[2])*npm.eye(len(Y)))

In [149]:
L

matrix([[  2.84206057e+000,   0.00000000e+000,   0.00000000e+000],
        [  2.07599043e-029,   2.84206057e+000,   0.00000000e+000],
        [  3.42464754e-300,   8.42436220e-184,   2.84206057e+000]])

In [151]:
Y = np.array([1,3,5])

In [152]:
Y.shape = (3,1)

In [153]:
Y

array([[1],
       [3],
       [5]])

In [157]:
np.linalg.solve(np.transpose(L),np.linalg.solve(L,Y))

array([[ 0.12380362],
       [ 0.37141086],
       [ 0.6190181 ]])

In [158]:
L

matrix([[  2.84206057e+000,   0.00000000e+000,   0.00000000e+000],
        [  2.07599043e-029,   2.84206057e+000,   0.00000000e+000],
        [  3.42464754e-300,   8.42436220e-184,   2.84206057e+000]])

In [159]:
X

array([[25, 15, -5],
       [15, 18,  0],
       [-5,  0, 11]])

In [161]:
def loglik(X,Y,hypers):
	#hypers as [length, signal std, noise std] in log space
	thing = X / sp.exp(hypers[0])
	dist = np.zeros((len(Y),len(Y)))
	for i in np.arange(len(Y)):
		for j in np.arange(len(Y)):
			dist[i,j] = np.sum(np.square(thing[i,:]-thing[j,:]))
	K = sp.exp(2*hypers[1]+(-1/2)*dist)
	L = np.linalg.cholesky(K)
	return (K,L)

In [165]:
(K,L)=loglik(X,Y,[0,1])

In [163]:
np.arange(0,3)

array([0, 1, 2])

In [164]:
np.linalg.cholesky(X)

array([[ 5.,  0.,  0.],
       [ 3.,  3.,  0.],
       [-1.,  1.,  3.]])

In [ ]:
	for i in np.arange(len(L)-1):
		for j in np.arange(i+1,len(L)):
			L[i,j] = L[j,i]